In [ ]:
import pandas as pd
import numpy as np
import ternary

C_olv_mol_wt = np.array([60.084, 79.866, 101.961, 71.846, 70.9374, 40.304, 56.079, 61.979, 94.2, 283.889])

data = pd.read_csv('Mafic Enclave Major Element data Mt. Shasta.csv')
data.columns = data.columns.str.strip()
data = data.drop(columns=['SAMPLE', 'Sum'])

final_compositions = []
initial_compositions = []
Qtz_final_list = []
Plag_final_list = []
Oliv_final_list = []
Qtz_initial_list = []
Plag_initial_list = []
Oliv_initial_list = []

for sample_name, row in data.iterrows():
    C_og_initial = np.array([
        row['SiO2'], row['TiO2'], row['Al2O3'], row['FeO*'], row['MnO'],
        row['MgO'], row['CaO'], row['Na2O'], row['K2O'], row['P2O5']
    ])
    C_og_initial = np.nan_to_num(C_og_initial)
   
    final_compositions.append(C_og_initial)
    initial_compositions.append(C_og_initial)

    Wt_Molar = np.array([1.003175789, 0.7923497268, 0.6210526316, 0.8816, 0.9047619048, 1.57082153, 1.128695652, 1.020408163, 0.6363636364, 1.00])
    conversion_factors = np.array([
    60.084 / 28.085,    # SiO2
    79.866 / 47.867,    # TiO2
    101.961 / (2 * 26.982), # Al2O3
    71.846 / 55.845,    # FeO
    70.9374 / 54.938,   # MnO
    40.304 / 24.305,    # MgO
    56.079 / 40.078,    # CaO
    61.979 / (2 * 22.990), # Na2O
    94.2 / 39.098,      # K2O
    283.889 / (2 * 30.974) # P2O5
    ])  
    MolarMonoxideComponents = conversion_factors / C_og_initial * 100
    MMC = MolarMonoxideComponents

    FMO = MMC[3] + MMC[5] 
    AlK = MMC[7] + MMC[8]
    SUM = MMC[0] - MMC[6] - 2*AlK + MMC[1] + 2*MMC[9]

    Qtz_final = (MMC[0] - 0.5*FMO - 1.5*MMC[6] - 0.25*MMC[2] - 2.75*AlK + 0.5*MMC[1] + 2.5*MMC[9]) / SUM
    Plag_final = (MMC[2] + MMC[7] - MMC[8]) / (2*SUM)
    Oliv_final = (FMO + 0.5*(MMC[2] - AlK) - MMC[6] - MMC[1] + 1.667*MMC[9])/(2*SUM)
    Cpx = (MMC[6] - 0.5*MMC[2] + 0.5*AlK - 1.667*MMC[9]) / SUM
    Ilm_Hem_Chr = (MMC[1]) / SUM
    Or = MMC[8] / SUM
    Ap = MMC[9] / (3*SUM)

    Qtz_final_list.append(Qtz_final)
    Plag_final_list.append(Plag_final)
    Oliv_final_list.append(Oliv_final)

    MolarMonoxideComponents_initial = C_og_initial / Wt_Molar / conversion_factors * 100
    MMC_initial = MolarMonoxideComponents_initial

    FMO_initial = MMC_initial[3] + MMC_initial[5]
    AlK_initial = MMC_initial[7] + MMC_initial[8]
    SUM_initial = (MMC_initial[0] + MMC_initial[1] + MMC_initial[2] + FMO_initial + MMC_initial[5] + MMC_initial[6] + MMC_initial[7] + MMC_initial[8] + MMC_initial[9])

    Qtz_initial = (MMC_initial[0] - 0.5 * FMO_initial - 1.5 * MMC_initial[6] - 0.25 * MMC_initial[2] - 2.75 * AlK_initial + 0.5 * MMC_initial[1] + 2.5 * MMC_initial[9]) / SUM_initial
    Plag_initial = (MMC_initial[2] + MMC_initial[7] - MMC_initial[8]) / (2*SUM_initial)
    Oliv_initial = (FMO_initial + 0.5 * (MMC_initial[2] - AlK_initial) - MMC_initial[6] - MMC_initial[1] + 1.667 * MMC_initial[9]) / (2 * SUM_initial)

    Qtz_initial_list.append(Qtz_initial)
    Plag_initial_list.append(Plag_initial)
    Oliv_initial_list.append(Oliv_initial)

#list -> arrays
Qtz_final_array = np.array(Qtz_final_list)
Plag_final_array = np.array(Plag_final_list)
Oliv_final_array = np.array(Oliv_final_list)

Qtz_initial_array = np.array(Qtz_initial_list)
Plag_initial_array = np.array(Plag_initial_list)
Oliv_initial_array = np.array(Oliv_initial_list)

#normalize
ternary_points_final = np.vstack((Qtz_final_array, Plag_final_array, Oliv_final_array)).T
ternary_points_final /= np.sum(ternary_points_final, axis=1)[:, np.newaxis]
ternary_points_initial = np.vstack((Qtz_initial_array, Plag_initial_array, Oliv_initial_array)).T
ternary_points_initial /= np.sum(ternary_points_initial, axis=1)[:, np.newaxis]

scale = 1 
figure, tax = ternary.figure(scale=scale)

tax.boundary(linewidth=2.0)
tax.gridlines(color="black", multiple=0.1)
tax.gridlines(color="blue", multiple=0.05, linewidth=0.5)

fontsize = 10
tax.set_title("Qtz_Plag-Olv", fontsize=fontsize)
tax.left_axis_label("Quartz", fontsize=fontsize)
tax.right_axis_label("Plagioclase", fontsize=fontsize)
tax.bottom_axis_label("Olivine", fontsize=fontsize)

tax.ticks(axis='lbr', linewidth=1)

tax.clear_matplotlib_ticks()

tax.scatter(ternary_points_final, marker='o', color='red', label="Final Sample Points")

tax.scatter(ternary_points_initial, marker='x', color='blue', label="Initial Sample Points")

tax.legend()

#ternary.plt.show()
